In [109]:
#loading the whole file
import pandas as pd 
df= pd.read_csv("train_set1.csv")

In [110]:
df

,code,code_tokens,docstring,docstring_tokens,func_name,language,original_string,partition,path,repo,sha,url
0,protected final void bindIndexed(Configuration...,"['protected', 'final', 'void', 'bindIndexed', ...",Bind indexed elements to the supplied collecti...,"['Bind', 'indexed', 'elements', 'to', 'the', '...",IndexedElementsBinder.bindIndexed,java,protected final void bindIndexed(Configuration...,train,spring-boot-project/spring-boot/src/main/java/...,spring-projects/spring-boot,0b27f7c70e164b2b1a96477f1d9c1acba56790c1,https://github.com/spring-projects/spring-boot...
1,public void setServletRegistrationBeans(\n\t\t...,"['public', 'void', 'setServletRegistrationBean...",Set {@link ServletRegistrationBean}s that the ...,"['Set', '{']",AbstractFilterRegistrationBean.setServletRegis...,java,public void setServletRegistrationBeans(\n\t\t...,train,spring-boot-project/spring-boot/src/main/java/...,spring-projects/spring-boot,0b27f7c70e164b2b1a96477f1d9c1acba56790c1,https://github.com/spring-projects/spring-boot...
2,public void addServletRegistrationBeans(\n\t\t...,"['public', 'void', 'addServletRegistrationBean...",Add {@link ServletRegistrationBean}s for the f...,"['Add', '{']",AbstractFilterRegistrationBean.addServletRegis...,java,public void addServletRegistrationBeans(\n\t\t...,train,spring-boot-project/spring-boot/src/main/java/...,spring-projects/spring-boot,0b27f7c70e164b2b1a96477f1d9c1acba56790c1,https://github.com/spring-projects/spring-boot...
3,public void setServletNames(Collection<String>...,"['public', 'void', 'setServletNames', '(', 'Co...",Set servlet names that the filter will be regi...,"['Set', 'servlet', 'names', 'that', 'the', 'fi...",AbstractFilterRegistrationBean.setServletNames,java,public void setServletNames(Collection<String>...,train,spring-boot-project/spring-boot/src/main/java/...,spring-projects/spring-boot,0b27f7c70e164b2b1a96477f1d9c1acba56790c1,https://github.com/spring-projects/spring-boot...
4,public void addServletNames(String... servletN...,"['public', 'void', 'addServletNames', '(', 'St...",Add servlet names for the filter.\n@param serv...,"['Add', 'servlet', 'names', 'for', 'the', 'fil...",AbstractFilterRegistrationBean.addServletNames,java,public void addServletNames(String... servletN...,train,spring-boot-project/spring-boot/src/main/java/...,spring-projects/spring-boot,0b27f7c70e164b2b1a96477f1d9c1acba56790c1,https://github.com/spring-projects/spring-boot...
...,...,...,...,...,...,...,...,...,...,...,...,...
454446,public static String primArrayToString(Object ...,"['public', 'static', 'String', 'primArrayToStr...",Returns the primitive array {@link Object#toSt...,"['Returns', 'the', 'primitive', 'array', '{', ...",Arrays2.primArrayToString,java,public static String primArrayToString(Object ...,train,benayn-ustyle/src/main/java/com/benayn/ustyle/...,jronrun/benayn,7585152e10e4cac07b4274c65f1c72ad7061ae69,https://github.com/jronrun/benayn/blob/7585152...
454447,public static <T> Object unwraps(T[] target) {...,"['public', 'static', '<', 'T', '>', 'Object', ...",Converts the given object array to primitive a...,"['Converts', 'the', 'given', 'object', 'array'...",Arrays2.unwraps,java,public static <T> Object unwraps(T[] target) {...,train,benayn-ustyle/src/main/java/com/benayn/ustyle/...,jronrun/benayn,7585152e10e4cac07b4274c65f1c72ad7061ae69,https://github.com/jronrun/benayn/blob/7585152...
454448,"static Object add(Object array, int index, Obj...","['static', 'Object', 'add', '(', 'Object', 'ar...",Adds the element at the specified position fro...,"['Adds', 'the', 'element', 'at', 'the', 'speci...",Arrays2.add,java,"static Object add(Object array, int index, Obj...",train,benayn-ustyle/src/main/java/com/benayn/ustyle/...,jronrun/benayn,7585152e10e4cac07b4274c65f1c72ad7061ae69,https://github.com/jronrun/benayn/blob/7585152...
454449,"public static Object remove(Object array, int ...","['public', 'static', 'Object', 'remove', '(', ...",Removes the element at the specified position ...,"['

In [111]:
import pandas as pd 
import string
import re
import pandas as pd 
import javalang
from javalang.ast import Node
import os
from anytree import AnyNode, RenderTree
import torch
import torch.utils.data as Data
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import transformers
import torch.optim as optim
import numpy as np
import random
import math
import time
from torch.nn.utils.rnn import pad_sequence
import string

In [112]:
#Generating abstract syntax tree
error=[]
dfset = df.drop([71],axis = 0)
for i, row in dfset.iterrows():
    try:
        programtext = row["code"]
        programtokens=javalang.tokenizer.tokenize(programtext)
        #print("programtokens",list(programtokens))
        parser=javalang.parse.Parser(programtokens)
        programast=parser.parse_member_declaration()
    except:
        error.append(i)
        continue
df_set = dfset.drop(error,axis = 0)

In [113]:
print(len(error))

398


In [114]:
df1 = df.drop(['code','docstring','func_name','language','original_string','partition','path','repo','sha','url'], axis=1)

In [119]:
#Removal of data with incomplete comments
for i, row in df1.iterrows():
    str1 = row['docstring_tokens']
    if '{' in str1:
        df1 = df1.drop([i],axis = 0)

In [1]:
#Removal of data with blank comments
for i, row in df1.iterrows():
    if row['docstring_tokens'] == '[]':
        df1 = df1.drop([i],axis = 0)

In [116]:
#Remove punctuation and split words
punctuation_string = string.punctuation
code_tokens=[]
doc_tokens=[]
for i, row in df1.iterrows():
    word = row['code_tokens']
    word1 = row['docstring_tokens']
    for j in punctuation_string:
        word = word.replace(j, '')
        word1 = word1.replace(j, '')
    word_list = word.split()
    doc_list = word1.split()
    list_alpha = []
    list_doc = []
    for word in word_list:
        word_split = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', word)).split()
        list_alpha.append(word_split)
    for word in doc_list:
        list_doc.append(word)
    list_alpha = [m for item in list_alpha for m in item]
    list_alpha = [item.lower() for item in list_alpha]
    list_doc = [item.lower() for item in list_doc]
    row['code_tokens'] = list_alpha
    row['docstring_tokens'] = list_doc
    code_tokens.append(list_alpha)
    doc_tokens.append(list_doc)

In [66]:
#convert string to list
punctuation_string = string.punctuation
for i, row in df1.iterrows():
    word1 = row['code_tokens']
    word2 = row['docstring_tokens']
    for j in punctuation_string:
        word1 = word1.replace(j, '')
        word2 = word2.replace(j, '')
    word_list1 = word1.split()
    word_list2 = word2.split()
    row['code_tokens'] = word_list1
    row['docstring_tokens'] = word_list2

In [17]:
#deleting data contains Chinese
def is_contain_chinese(check_str):
    for ch in check_str:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False
for i, row in df1.iterrows():
    str1 = "".join(row['docstring_tokens'])
    if is_contain_chinese(str1):
        df1 = df1.drop([i],axis = 0)

In [42]:
df1 = df1.drop(['Unnamed: 0'],axis = 1)

In [117]:
df1

,code_tokens,docstring_tokens
0,"[protected, final, void, bind, indexed, config...","[bind, indexed, elements, to, the, supplied, c..."
3,"[public, void, set, servlet, names, collection...","[set, servlet, names, that, the, filter, will,..."
4,"[public, void, add, servlet, names, string, se...","[add, servlet, names, for, the, filter]"
5,"[public, void, set, url, patterns, collection,...","[set, the, url, patterns, that, the, filter, w..."
6,"[public, void, add, url, patterns, string, url...","[add, url, patterns, as, defined, in, the, ser..."
...,...,...
454446,"[public, static, string, prim, array, to, stri...","[returns, the, primitive, array, link, objectt..."
454447,"[public, static, t, object, unwraps, t, target...","[converts, the, given, object, array, to, prim..."
454448,"[static, object, add, object, array, int, inde...","[adds, the, element, at, the, specified, posit..."
454449,"[public, static, object, remove, object, array...","[removes, the, element, at, the, specified, po..."


In [120]:
list_index = list(df1.index)

In [121]:
#build a class to split the dataset
def split_ids(id_list, train=.7, valid=0.1, test=0.2):
    
    list_copy = id_list.copy()
    random.shuffle(list_copy)
    
    train_size = math.floor(len(list_copy) * train)
    valid_size = math.floor(len(list_copy) * valid)
    
    return list_copy[:train_size], list_copy[train_size:(train_size + valid_size)], list_copy[(train_size + valid_size):]



In [122]:
train_set, valid_set, test_set = split_ids(list_index)

In [123]:
print(len(train_set),len(valid_set),len(test_set))

297584 42512 85025


In [124]:
df_train = df1.loc[train_set]
df_valid = df1.loc[valid_set]
df_test = df1.loc[test_set]

In [125]:
df_train.to_csv('df_train2.csv')
df_valid.to_csv('df_valid2.csv')
df_test.to_csv('df_test2.csv')

In [126]:
df_train = pd.read_csv('df_train2.csv')

In [129]:
#Test the model on 1000 pieces of data
df_train = df_train.head(1000)

In [132]:
#str to list
punctuation_string = string.punctuation
for i, row in df_train.iterrows():
    word1 = row['code_tokens']
    word2 = row['docstring_tokens']
    for j in punctuation_string:
        word1 = word1.replace(j, '')
        word2 = word2.replace(j, '')
    word_list1 = word1.split()
    word_list2 = word2.split()
    row['code_tokens'] = word_list1
    row['docstring_tokens'] = word_list2

In [133]:
def lengthCheck(df):
    list1=[]
    list2=[]
    for i, row in df.iterrows():
        length1 = len(df.iloc[i].iat[0])
        length2 = len(df.iloc[i].iat[1])
        list1.append(length1)
        list2.append(length2)
    return list1,list2

In [134]:
length_list1,length_list2 = lengthCheck(df_train)

In [135]:
#determine 200 is the fixed length
num = sum(i <200 for i in length_list1)
print(num)
print(len(length_list1))
num = sum(i <200 for i in length_list2)
print(num)
print(len(length_list2))

910
1000
998
1000


In [136]:
def cutLength(df):
    for i, row in df.iterrows():
        length1 = len(df.iloc[i].iat[0])
        length2 = len(df.iloc[i].iat[1])
        a = df.iloc[i].iat[0]
        b = df.iloc[i].iat[1]
        if length1>200:
            del a[200:]
        if length2>200:
             del b[200:]
    return df

In [137]:
df_train_cut = cutLength(df_train)

In [139]:
#delete the tokens have appeared less than 3 times
list_x = []
list_Y = []
for i, row in df_train_cut.iterrows():
    list_x.append(row['code_tokens'])
    list_Y.append(row['docstring_tokens'])
list_a = [m for item in list_x for m in item]
list_b = [m for item in list_Y for m in item]

In [140]:
from collections import Counter
count = Counter(list_a)
word_a = [x[0] for x in count.items() if x[1] > 3]
print(len(word_a))
count1 = Counter(list_b)
word_b = [x[0] for x in count1.items() if x[1] > 3]
print(len(word_b))

2059
610


In [38]:
#build dictionary
size1 = len(word_a)
size2 = len(word_b)
tokenids = range(3,size1+3)
docids = range(3,size2+3)
src_dict = dict(zip(word_a, tokenids))
trg_dict = dict(zip(word_b, docids))
src_dict['<sos>']=0
src_dict['<eos>']=1
src_dict['<unk>']=2
trg_dict['<sos>']=0
trg_dict['<eos>']=1
trg_dict['<unk>']=2

In [40]:
#Mapping data through the dictionary
for i in list_trg:
    list_vocab_a=[]
    list_vocab_a.append(trg_dict['<sos>'])
    for m in i:
        if not m in trg_dict.keys():
            list_vocab_a.append(trg_dict['<unk>'])
        else:
            list_vocab_a.append(trg_dict[m])
    list_vocab_a.append(trg_dict['<eos>'])
    dic_trg.append(list_vocab_a)

In [41]:
print(len(src_dict),len(trg_dict))


57 23


In [42]:
trg_dict['ignore_idx']=len(trg_dict)

In [43]:
class Data_Subset(Dataset):

    def __init__(self,s1,s2,ignore):
        self.s1 = s1
        self.s2 = s2
        self.src = s1
        self.trg = s2
        self.ig = ignore
    def __getitem__(self, index):
        
        while len(self.src[index])< 202:
            self.src[index].append(1)
        
        while len(self.trg[index])< 202:
            self.trg[index].append(self.ig)
        return torch.LongTensor(self.src[index]), torch.LongTensor(self.trg[index])
    def __len__(self):
        return len(self.s1)
    
dataset_train = Data_Subset(dic_src, dic_trg,trg_dict['ignore_idx'])

In [44]:
train_loader = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=5,  
        shuffle=True,
        num_workers=0, 
    )

In [45]:
train_iter = iter(train_loader)
a,b = train_iter.next()
print(a.shape)
print(b.shape)

torch.Size([5, 202])
torch.Size([5, 202])


In [64]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
       
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell


In [65]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell


In [68]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
    
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [69]:
INPUT_DIM = len(src_dict)
OUTPUT_DIM = len(trg_dict)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
device = torch.device('cpu')
model = Seq2Seq(enc, dec, device).to(device)

In [70]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(57, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(24, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=24, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [71]:
optimizer = optim.Adam(model.parameters())

In [72]:
criterion = nn.CrossEntropyLoss(ignore_index = len(trg_dict))

In [73]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, (src,trg) in enumerate(iterator):
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [74]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    
    print(train_loss)

3.024348735809326
1.8447325825691223
0.7550484240055084
1.396475613117218
0.7638779729604721
0.8837899267673492
0.7894061207771301
0.6787824630737305
0.7395245730876923
0.8383639752864838
